# YOLO Model Comparison: Vanilla vs Fine-tuned Bosphorus Model

This notebook compares the performance of vanilla YOLO models (YOLOv8n, YOLOv8s, YOLOv11n, YOLOv11s) against a fine-tuned Bosphorus ship detection model on a custom test dataset.

**Metrics Evaluated:**
- Detection count vs Ground Truth
- Intersection over Union (IoU)
- Average confidence score
- Inference time (ms)

**Output:** Comparison metrics (CSV), visual detection grids, and performance charts.

## Step 1: Setup and Configuration

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os

# Paths
DATASET_PATH = "/content/drive/MyDrive/Uskudar Uni Master/YOLO Model klasor/Test Dataset"
FINETUNED_MODEL_PATH = "/content/drive/MyDrive/Uskudar Uni Master/YOLO Model klasor/runs/bosphorus_yolo11s_10884/weights/best.pt"
OUTPUT_DIR = "/content/drive/MyDrive/Uskudar Uni Master/YOLO Model klasor/Comparisons"

# Test paths (images and labels directly under Test Dataset folder)
TEST_IMAGES_PATH = os.path.join(DATASET_PATH, "images")
TEST_LABELS_PATH = os.path.join(DATASET_PATH, "labels")

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Verify paths
print("Path Verification:")
print(f"  Dataset: {'OK' if os.path.exists(DATASET_PATH) else 'MISSING'}")
print(f"  Fine-tuned model: {'OK' if os.path.exists(FINETUNED_MODEL_PATH) else 'MISSING'}")
print(f"  Test images: {'OK' if os.path.exists(TEST_IMAGES_PATH) else 'MISSING'}")
print(f"  Test labels: {'OK' if os.path.exists(TEST_LABELS_PATH) else 'MISSING'}")
print(f"  Output directory: {OUTPUT_DIR}")

In [ ]:
# Install Ultralytics
!pip install -U ultralytics

from ultralytics import YOLO
import ultralytics
print(f"Ultralytics version: {ultralytics.__version__}")

In [ ]:
# Check GPU
import torch

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("No GPU detected - go to Runtime > Change runtime type > GPU")

## Step 2: Load Models

In [ ]:
# Load models
from ultralytics import YOLO

# ============================================================
# VANILLA MODELS TO COMPARE (add/remove as needed)
# ============================================================
VANILLA_MODELS = {
    'YOLOv8n': 'yolov8n.pt',   # Nano - fastest, smallest
    'YOLOv8s': 'yolov8s.pt',   # Small
    'YOLOv11n': 'yolo11n.pt',  # Nano v11
    'YOLOv11s': 'yolo11s.pt',  # Small v11 (same arch as fine-tuned)
}
# ============================================================

# Load vanilla models
vanilla_models = {}
for name, weights in VANILLA_MODELS.items():
    print(f"Loading {name}...", end=" ")
    vanilla_models[name] = YOLO(weights)
    print(f"OK ({len(vanilla_models[name].names)} COCO classes)")

# Fine-tuned Bosphorus model
print(f"\nLoading Fine-tuned Bosphorus model...", end=" ")
finetuned_model = YOLO(FINETUNED_MODEL_PATH)
print(f"OK")
print(f"  Classes: {finetuned_model.names}")

## Step 3: Select Test Images

In [ ]:
# Select random test images
import random
import os

NUM_IMAGES = 8  # Select 5-10 images

# Get all test images
all_test_images = [f for f in os.listdir(TEST_IMAGES_PATH) if f.endswith(('.jpg', '.jpeg', '.png'))]
print(f"Total test images: {len(all_test_images)}")

# Random selection
random.seed(42)  # For reproducibility
selected_images = random.sample(all_test_images, min(NUM_IMAGES, len(all_test_images)))

print(f"\nSelected {len(selected_images)} images for comparison:")
for img in selected_images:
    print(f"  - {img}")

## Step 4: Helper Functions

In [ ]:
import numpy as np
import time
import cv2

def parse_yolo_label(label_path, img_width, img_height):
    """Parse YOLO label file to absolute coordinates."""
    boxes = []
    if not os.path.exists(label_path):
        return boxes
    
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                x_center = float(parts[1]) * img_width
                y_center = float(parts[2]) * img_height
                width = float(parts[3]) * img_width
                height = float(parts[4]) * img_height
                
                x1 = x_center - width / 2
                y1 = y_center - height / 2
                x2 = x_center + width / 2
                y2 = y_center + height / 2
                boxes.append([x1, y1, x2, y2])
    return boxes

def calculate_iou(box1, box2):
    """Calculate IoU between two boxes."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0

def compute_avg_iou(pred_boxes, gt_boxes):
    """Compute average best IoU for predictions against ground truth (only matched boxes)."""
    if not pred_boxes or not gt_boxes:
        return 0.0
    best_ious = [max(calculate_iou(pred, gt) for gt in gt_boxes) for pred in pred_boxes]
    # Only average IoUs > 0 (matched predictions)
    matched_ious = [iou for iou in best_ious if iou > 0]
    return np.mean(matched_ious) if matched_ious else 0.0

def compute_detection_metrics(pred_boxes, gt_boxes, iou_threshold=0.5):
    """
    Compute precision, recall, and matched IoU using Hungarian matching.
    
    Returns:
        dict with 'precision', 'recall', 'avg_matched_iou', 'true_positives', 'false_positives', 'false_negatives'
    """
    if not pred_boxes and not gt_boxes:
        return {'precision': 1.0, 'recall': 1.0, 'avg_matched_iou': 1.0, 'tp': 0, 'fp': 0, 'fn': 0}
    if not pred_boxes:
        return {'precision': 0.0, 'recall': 0.0, 'avg_matched_iou': 0.0, 'tp': 0, 'fp': 0, 'fn': len(gt_boxes)}
    if not gt_boxes:
        return {'precision': 0.0, 'recall': 0.0, 'avg_matched_iou': 0.0, 'tp': 0, 'fp': len(pred_boxes), 'fn': 0}
    
    # Greedy matching: for each prediction, find best GT match
    gt_matched = [False] * len(gt_boxes)
    matched_ious = []
    tp = 0
    
    for pred in pred_boxes:
        best_iou = 0
        best_gt_idx = -1
        for gt_idx, gt in enumerate(gt_boxes):
            if gt_matched[gt_idx]:
                continue
            iou = calculate_iou(pred, gt)
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = gt_idx
        
        if best_iou >= iou_threshold and best_gt_idx >= 0:
            gt_matched[best_gt_idx] = True
            matched_ious.append(best_iou)
            tp += 1
    
    fp = len(pred_boxes) - tp
    fn = len(gt_boxes) - tp
    
    precision = tp / len(pred_boxes) if pred_boxes else 0.0
    recall = tp / len(gt_boxes) if gt_boxes else 0.0
    avg_matched_iou = np.mean(matched_ious) if matched_ious else 0.0
    
    return {
        'precision': precision,
        'recall': recall, 
        'avg_matched_iou': avg_matched_iou,
        'tp': tp,
        'fp': fp,
        'fn': fn
    }

def run_inference(model, image_path, imgsz, conf, classes=None):
    """
    Run inference with specified parameters.
    
    Args:
        model: YOLO model
        image_path: Path to image
        imgsz: Image size for inference
        conf: Confidence threshold
        classes: List of class indices to filter (None = all classes)
    """
    start_time = time.time()
    results = model.predict(source=image_path, conf=conf, imgsz=imgsz, classes=classes, verbose=False)
    inference_time = (time.time() - start_time) * 1000
    
    boxes = [box.xyxy[0].cpu().numpy().tolist() for r in results for box in r.boxes]
    confidences = [float(box.conf[0]) for r in results for box in r.boxes]
    
    return {
        'boxes': boxes,
        'confidences': confidences,
        'num_detections': len(boxes),
        'avg_confidence': np.mean(confidences) if confidences else 0.0,
        'inference_time_ms': inference_time,
        'results': results
    }

print("Helper functions loaded!")

## Step 5: Run Comparison

In [ ]:
import pandas as pd

# ============================================================
# MODEL PARAMETERS
# ============================================================
# Vanilla models: DEFAULT/MINIMAL parameters (NO optimization)
VANILLA_IMGSZ = 640      # Default YOLO size
VANILLA_CONF = 0.25
VANILLA_CLASSES = None   # No class filtering - detect everything

# Fine-tuned: OPTIMAL parameters (as trained)
FINETUNED_IMGSZ = 1088   # Optimal for 1920x1080 images
FINETUNED_CONF = 0.25
FINETUNED_CLASSES = None
# ============================================================

# Store results: {model_name: [per_image_results]}
all_results = {name: [] for name in vanilla_models.keys()}
all_results['Fine-tuned'] = []

IOU_THRESHOLD = 0.5  # Standard threshold for matching

print("Running comparison...")
print(f"  Vanilla models: imgsz={VANILLA_IMGSZ}, conf={VANILLA_CONF}")
print(f"  Fine-tuned:     imgsz={FINETUNED_IMGSZ}, conf={FINETUNED_CONF}")
print(f"  IoU threshold:  {IOU_THRESHOLD}")
print()

for img_idx, img_name in enumerate(selected_images):
    img_path = os.path.join(TEST_IMAGES_PATH, img_name)
    label_path = os.path.join(TEST_LABELS_PATH, os.path.splitext(img_name)[0] + '.txt')
    
    img = cv2.imread(img_path)
    img_height, img_width = img.shape[:2]
    gt_boxes = parse_yolo_label(label_path, img_width, img_height)
    
    print(f"[{img_idx+1}/{len(selected_images)}] {img_name} (GT: {len(gt_boxes)} ships)")
    
    # Run all vanilla models
    for name, model in vanilla_models.items():
        results = run_inference(model, img_path, VANILLA_IMGSZ, VANILLA_CONF, VANILLA_CLASSES)
        metrics = compute_detection_metrics(results['boxes'], gt_boxes, IOU_THRESHOLD)
        all_results[name].append({
            'image': img_name,
            'gt_count': len(gt_boxes),
            'detections': results['num_detections'],
            'avg_conf': results['avg_confidence'],
            'precision': metrics['precision'],
            'recall': metrics['recall'],
            'avg_matched_iou': metrics['avg_matched_iou'],
            'tp': metrics['tp'],
            'fp': metrics['fp'],
            'fn': metrics['fn'],
            'time_ms': results['inference_time_ms'],
            'raw': results
        })
        print(f"    {name}: {results['num_detections']} det, P={metrics['precision']:.2f}, R={metrics['recall']:.2f}, IoU={metrics['avg_matched_iou']:.2f}, {results['inference_time_ms']:.0f}ms")
    
    # Run fine-tuned model
    results = run_inference(finetuned_model, img_path, FINETUNED_IMGSZ, FINETUNED_CONF, FINETUNED_CLASSES)
    metrics = compute_detection_metrics(results['boxes'], gt_boxes, IOU_THRESHOLD)
    all_results['Fine-tuned'].append({
        'image': img_name,
        'gt_count': len(gt_boxes),
        'detections': results['num_detections'],
        'avg_conf': results['avg_confidence'],
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'avg_matched_iou': metrics['avg_matched_iou'],
        'tp': metrics['tp'],
        'fp': metrics['fp'],
        'fn': metrics['fn'],
        'time_ms': results['inference_time_ms'],
        'raw': results
    })
    print(f"    Fine-tuned: {results['num_detections']} det, P={metrics['precision']:.2f}, R={metrics['recall']:.2f}, IoU={metrics['avg_matched_iou']:.2f}, {results['inference_time_ms']:.0f}ms")

print("\nComparison complete!")

## Step 6: Generate Metrics Table

In [ ]:
# Generate summary statistics for all models
model_names = list(all_results.keys())
gt_total = sum(r['gt_count'] for r in all_results['Fine-tuned'])

summary_stats = {}
for name in model_names:
    results = all_results[name]
    total_det = sum(r['detections'] for r in results)
    total_tp = sum(r['tp'] for r in results)
    total_fp = sum(r['fp'] for r in results)
    total_fn = sum(r['fn'] for r in results)
    
    # Calculate overall precision and recall
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    
    # Average matched IoU (only for true positives)
    matched_ious = [r['avg_matched_iou'] for r in results if r['tp'] > 0]
    avg_matched_iou = np.mean(matched_ious) if matched_ious else 0.0
    
    avg_conf = np.mean([r['avg_conf'] for r in results if r['avg_conf'] > 0]) if any(r['avg_conf'] > 0 for r in results) else 0
    avg_time = np.mean([r['time_ms'] for r in results])
    
    summary_stats[name] = {
        'Total Detections': total_det,
        'TP': total_tp,
        'FP': total_fp,
        'FN': total_fn,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Avg Matched IoU': avg_matched_iou,
        'Avg Confidence': avg_conf,
        'Avg Time (ms)': avg_time
    }

# Create summary DataFrame
summary_df = pd.DataFrame(summary_stats).T
summary_df['Ground Truth'] = gt_total
summary_df = summary_df[['Ground Truth', 'Total Detections', 'TP', 'FP', 'FN', 'Precision', 'Recall', 'F1 Score', 'Avg Matched IoU', 'Avg Confidence', 'Avg Time (ms)']]

# Format for display
display_df = summary_df.copy()
display_df['Precision'] = display_df['Precision'].apply(lambda x: f"{x:.2%}")

display_df['Recall'] = display_df['Recall'].apply(lambda x: f"{x:.2%}")print(f"  Fastest:           {summary_df['Avg Time (ms)'].idxmin()} ({summary_df['Avg Time (ms)'].min():.1f}ms)")

display_df['F1 Score'] = display_df['F1 Score'].apply(lambda x: f"{x:.2%}")print(f"  Best Confidence:   {summary_df['Avg Confidence'].idxmax()} ({summary_df['Avg Confidence'].max():.2%})")

display_df['Avg Matched IoU'] = display_df['Avg Matched IoU'].apply(lambda x: f"{x:.3f}")print(f"  Best Matched IoU:  {summary_df['Avg Matched IoU'].idxmax()} ({summary_df['Avg Matched IoU'].max():.3f})")

display_df['Avg Confidence'] = display_df['Avg Confidence'].apply(lambda x: f"{x:.2%}")print(f"  Best Recall:       {summary_df['Recall'].idxmax()} ({summary_df['Recall'].max():.2%})")

display_df['Avg Time (ms)'] = display_df['Avg Time (ms)'].apply(lambda x: f"{x:.1f}")print(f"  Best Precision:    {summary_df['Precision'].idxmax()} ({summary_df['Precision'].max():.2%})")

print(f"  Best F1 Score:     {summary_df['F1 Score'].idxmax()} ({summary_df['F1 Score'].max():.2%})")

print("=" * 120)print("=" * 120)

print(f"SUMMARY: ALL MODELS COMPARISON (IoU threshold = {IOU_THRESHOLD})")print("BEST MODEL PER METRIC:")

print("=" * 120)print("\n" + "=" * 120)

display(display_df)# Find best model for each metric


In [ ]:
# Save metrics to CSV
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Per-image results for all models
rows = []
for img_idx in range(len(selected_images)):
    row = {'image': all_results['Fine-tuned'][img_idx]['image'], 
           'gt_count': all_results['Fine-tuned'][img_idx]['gt_count']}
    for name in model_names:
        r = all_results[name][img_idx]
        row[f'{name}_detections'] = r['detections']
        row[f'{name}_tp'] = r['tp']
        row[f'{name}_fp'] = r['fp']
        row[f'{name}_fn'] = r['fn']
        row[f'{name}_precision'] = r['precision']
        row[f'{name}_recall'] = r['recall']
        row[f'{name}_matched_iou'] = r['avg_matched_iou']
        row[f'{name}_conf'] = r['avg_conf']
        row[f'{name}_time_ms'] = r['time_ms']
    rows.append(row)

print(f"Metrics saved to: {csv_path}")

csv_df = pd.DataFrame(rows)csv_df.to_csv(csv_path, index=False)
csv_path = os.path.join(OUTPUT_DIR, f"comparison_metrics_{timestamp}.csv")

## Step 7: Visual Comparison

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

def create_comparison_grid(all_results, model_names, output_path, max_images=4):
    """Create comparison grid for all models."""
    n_images = min(len(all_results['Fine-tuned']), max_images)
    n_models = len(model_names)
    
    fig = plt.figure(figsize=(5 * n_models, 5 * n_images))
    gs = GridSpec(n_images, n_models, figure=fig, hspace=0.3, wspace=0.1)
    
    for img_idx in range(n_images):
        for model_idx, name in enumerate(model_names):
            result = all_results[name][img_idx]
            annotated = cv2.cvtColor(result['raw']['results'][0].plot(), cv2.COLOR_BGR2RGB)
            
            ax = fig.add_subplot(gs[img_idx, model_idx])
            ax.imshow(annotated)
            
            imgsz = FINETUNED_IMGSZ if name == 'Fine-tuned' else VANILLA_IMGSZ
            ax.set_title(f"{name} (imgsz={imgsz})\n{result['detections']} det | IoU: {result['iou']:.2f}", 
                        fontsize=9, fontweight='bold')
            ax.axis('off')
            
            if model_idx == 0:
                ax.text(-0.02, 0.5, f"{result['image'][:20]}...\n(GT: {result['gt_count']})", 
                       transform=ax.transAxes, fontsize=8, va='center', ha='right', fontweight='bold')
    
    fig.suptitle('Model Comparison: Vanilla Models (Default Params) vs Fine-tuned (Optimal Params)', 
                 fontsize=12, fontweight='bold', y=1.02)
    
    plt.savefig(output_path, dpi=150, bbox_inches='tight', facecolor='white')
    plt.show()
    print(f"\nComparison grid saved to: {output_path}")

grid_path = os.path.join(OUTPUT_DIR, f"comparison_grid_{timestamp}.png")
create_comparison_grid(all_results, model_names, grid_path)

## Step 8: Speed Comparison Chart

In [ ]:
import matplotlib.pyplot as plt

# Colors for each model
colors = plt.cm.tab10(np.linspace(0, 1, len(model_names)))

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Inference Time Comparison (bar chart)
ax1 = axes[0]
times = [summary_stats[name]['Avg Time (ms)'] for name in model_names]
bars = ax1.bar(model_names, times, color=colors, edgecolor='black', linewidth=1)
ax1.set_ylabel('Avg Inference Time (ms)', fontsize=11)
ax1.set_title('Inference Speed', fontsize=12, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)
for bar, t in zip(bars, times):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{t:.0f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# 2. Average IoU Comparison (bar chart)
ax2 = axes[1]
ious = [summary_stats[name]['Avg IoU'] for name in model_names]
bars = ax2.bar(model_names, ious, color=colors, edgecolor='black', linewidth=1)
ax2.set_ylabel('Average IoU', fontsize=11)
ax2.set_title('Detection Quality (IoU)', fontsize=12, fontweight='bold')
ax2.set_ylim(0, 1.0)
ax2.tick_params(axis='x', rotation=45)
for bar, iou in zip(bars, ious):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{iou:.2f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# 3. Detection Count vs Ground Truth
ax3 = axes[2]
x = np.arange(len(selected_images))
width = 0.8 / len(model_names)

for i, name in enumerate(model_names):
    dets = [all_results[name][j]['detections'] for j in range(len(selected_images))]
    ax3.bar(x + i * width - 0.4 + width/2, dets, width, label=name, color=colors[i], alpha=0.8)

gt_counts = [all_results['Fine-tuned'][j]['gt_count'] for j in range(len(selected_images))]
ax3.plot(x, gt_counts, 'ko-', linewidth=2, markersize=6, label='Ground Truth')

ax3.set_ylabel('Detection Count', fontsize=11)
ax3.set_xlabel('Image', fontsize=11)
ax3.set_title('Detections per Image', fontsize=12, fontweight='bold')
ax3.set_xticks(x)
ax3.set_xticklabels([f'{i+1}' for i in range(len(selected_images))])
ax3.legend(loc='upper right', fontsize=8)

plt.tight_layout()

speed_chart_path = os.path.join(OUTPUT_DIR, f"comparison_charts_{timestamp}.png")
plt.savefig(speed_chart_path, dpi=150, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Charts saved to: {speed_chart_path}")

## Step 9: Summary Report

In [ ]:
# Final Summary
print("=" * 90)
print("MULTI-MODEL COMPARISON SUMMARY")
print("=" * 90)

print("\nPARAMETER CONFIGURATION:")
print(f"  Vanilla models:       imgsz={VANILLA_IMGSZ} (default), conf={VANILLA_CONF}, classes=all")
print(f"  Fine-tuned Bosphorus: imgsz={FINETUNED_IMGSZ} (optimal), conf={FINETUNED_CONF}, classes=all")
print(f"\nTest Images: {len(selected_images)} | Ground Truth Ships: {gt_total}")

print("\n" + "-" * 90)
print(f"{'Model':<15} {'Detections':<12} {'Avg IoU':<10} {'Avg Conf':<12} {'Avg Time':<12} {'Notes'}")
print("-" * 90)

for name in model_names:
    s = summary_stats[name]
    notes = "⭐ BEST IoU" if name == summary_df['Avg IoU'].idxmax() else ""
    notes += " 🚀 FASTEST" if name == summary_df['Avg Time (ms)'].idxmin() else ""
    print(f"{name:<15} {s['Total Detections']:<12} {s['Avg IoU']:<10.3f} {s['Avg Confidence']:<12.2%} {s['Avg Time (ms)']:<12.1f} {notes}")

print("-" * 90)
print(f"{'Ground Truth':<15} {gt_total:<12}")
print("=" * 90)

print("\nOUTPUT FILES:")
print(f"  - Metrics: {csv_path}")
print(f"  - Visual:  {grid_path}")
print(f"  - Charts:  {speed_chart_path}")
print("=" * 90)

In [ ]:
# Verify all output files exist
print("\nOutput files verification:")
for filepath in [csv_path, grid_path, speed_chart_path]:
    if os.path.exists(filepath):
        size_kb = os.path.getsize(filepath) / 1024
        print(f"  ✓ {os.path.basename(filepath)} ({size_kb:.1f} KB)")
    else:
        print(f"  ✗ {os.path.basename(filepath)} - MISSING")